In [1]:
import google.generativeai as genai

# 1️ Configure your API key
genai.configure(api_key="")

# 2️ Create a model handle (remote model on Google servers)
model = genai.GenerativeModel("gemini-1.5-flash")

# 3️ Quick test with a tiny text prompt
response = model.generate_content([{"text": "Hello, can you give a short JSON response like {'test': 5}?"}])

# 4️ Print the response
print(response.text)


```json
{"test": 5}
```



In [16]:
import os
import json
import glob
import pandas as pd
import google.generativeai as genai

# Configure Gemini
genai.configure(api_key="")

# Paths
root = "/content/drive/MyDrive/eval_images"
folder_name = "Flux-Dev_drawbench_images"
image_index = 188
baseline_file = "/content/drive/MyDrive/DrawBenchPrompts.csv"

# Load baseline prompts
baseline_df = pd.read_csv(baseline_file)
baseline_prompt = baseline_df.loc[image_index, "Prompts"]

# Get actual image name from CSV
image_name = str(baseline_df.loc[image_index, "image_name"])  # Convert to string if not

# Find image with any extension
image_pattern = os.path.join(root, folder_name, f"{image_name}.*")
image_files = glob.glob(image_pattern)
if not image_files:
    raise FileNotFoundError(f"No image found for {image_name} in {folder_name}")
image_path = image_files[0]

# Read image bytes
with open(image_path, "rb") as f:
    image_bytes = f.read()

# Build updated evaluation prompt
prompt = f"""
You are a strict evaluator. Given a baseline text prompt and an image, judge if the image reflects the baseline.

Rate each dimension on a scale of 0–5 (integers only):
1. Relevance: Does the image content correspond to the baseline prompt?
2. Color Accuracy: Are the colors in the image consistent with the baseline prompt?
3. Shape Matching: Are the shapes of objects correct and recognizable with respect to baseline prompt?
4. Number of Objects: Does the image contain the correct number of objects described in the baseline prompt If yes return Correct else Incorrect?
5. Text Correctness: If the baseline prompt contains any text inside '' check whether the image contains that text or not, else return None?
6. Positioning: Are the objects positioned as described in the baseline prompt?

Respond strictly in JSON:
{{
  "relevance": X,
  "color_accuracy": Y,
  "shape_matching": Z,
  "number_of_objects": A,
  "text_correctness": B,
  "positioning": C
}}

Baseline prompt: "{baseline_prompt}"
"""

# Load Gemini model
model = genai.GenerativeModel("gemini-1.5-flash")

# Generate evaluation
response = model.generate_content([
    prompt,
    {"mime_type": "image/jpeg", "data": image_bytes}
])

# Print raw response first
print("Raw response text:")
print(response.text)

# Parse JSON
try:
    result = json.loads(response.text.strip())
    result["index"] = image_index
    print("Parsed JSON result:")
    print(json.dumps(result, indent=4))
except Exception as e:
    print("JSON parsing failed:", e)


Raw response text:
```json
{
  "relevance": 1,
  "color_accuracy": 4,
  "shape_matching": 5,
  "number_of_objects": "Correct",
  "text_correctness": 0,
  "positioning": 5
}
```
JSON parsing failed: Expecting value: line 1 column 1 (char 0)
